<a href="https://colab.research.google.com/github/Yunpei24/BigDataBase/blob/main/Programmation_sous_MapReduceTP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction à la programmation distribuée sous <i>MapReduce</i>
L'objet principal de ce notebook est de maîtriser la programmation de traitement distribué sous <b>MapReduce</b>. 
Pour rappel, sous Hadoop, il ne revient pas au programmeur applicatif de mettre en œuvre les mécanismes de réplication, ni de gérer la traçabilité des grains et leur réaffectation. Ces tâches sont de la responsabilité du framework. 

Le programmeur applicatif a néanmoins le rôle de reformuler les algorithmes qu'il souhaite mettre en œuvre sur la plateforme distribuée suivant le mécanisme d’exécution <i>MapReduce</i>, un mécanisme d’exécution très populaire, présent dans plusieurs frameworks distribués.

<i>MapReduce</i> décompose l’ensemble des opérations à réaliser en deux types de tâches <b>élémentaires</b> et <b>uniformes</b>, les <i>Map</i> et les <i>Reduce</i>. Chaque donnée passe d'abord par une tâche <i>Map</i> qui la transforme et éventuellement par une seconde tâche <i>Reduce</i>. 

Aucun ordre d'exécution particulier n'est attendu entre différentes tâches <i>Map</i> ou entre différentes tâches <i>Reduce</i>. Une ou plusieurs tâches <i>Map</i> et/ou <i>Reduce</i> peuvent être facilement assignées à chaque nœud de calcul. Un nœud de calcul peut correspondre à un ordinateur individuel ou à un cœur d'une unité centrale multi-cœur. Dans ce dernier cas, la mémoire vive et le stockage de masse de l’ordinateur sont partagés entre les cœurs.

Le fonctionnement général de MapReduce est constitué des étapes suivantes:
<ol>
<li>L'ensemble de données à traiter est découpé en fragments (<i>chunks</i>).</li>
<li>Chaque tâche <i>Map</i> est assignée à un nœud de calcul qui reçoit un ou plusieurs fragments que la tâche <i>Map</i> transforme en une séquence de paires \[clé, valeur].</li>
<li>Chaque tâche <i>Reduce</i> est associée à une ou plusieurs clés et est assignée à un nœud de calcul.</li>
<li>Les paires (clé, valeur) produites par les <i>Map</i> sont groupées par clés et stockées sur les nœuds de calcul qui exécuteront les tâches <i>Reduce</i> respectives (étape shuffle).</li>
<li>Chaque tâche <i>Reduce</i> combine, pour chaque clé qui lui est associée, les valeurs des paires [clé, valeur] avec cette clé ; les résultats sont stockés et constituent le résultat du traitement.</li>
</ol>

Le programmeur écrit les fonctions <i>Map</i> et <i>Reduce</i>, le framework se charge du reste comme illustré ci-dessous dans le décompte distribué de la fréquence de chaque mot d'un corpus.
<!-- img width="70%" src="https://res.cloudinary.com/talend/image/upload/q_auto,w_923,h_486/resources/seo-articles/seo-what-is-mapreduce_gj9ehi.webp" -->

<img width="70%" src="https://www.nayaa.fr/bigdata/mr-execution-ex.png">

##Installation du Java Development Kit (JDK) 
Hadoop est écrit en Java et nécessite donc l'installation d'exécution de Java.

Installation du JDK

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Création de la variable d'environnement <JAVA_HOME> pour situer l'emplacement d'installationde Java 

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Installation du framework Hadoop

Téléchargement depuis les archives de la fondation Apache

In [ ]:
!wget https://archive.apache.org/dist/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz

Extraction de l'archive

In [ ]:
!tar -xzvf hadoop-3.3.0.tar.gz

Copie du dossier extrait dans l'emplacement <user/local>

In [5]:
!cp -r hadoop-3.3.0/ /usr/local/

## Programmation de tâches distribuées avec <i>MapReduce</i>

Création d'un repertoire <myinput> pour contenir le jeu de données à tester durant cet exercice e d'un second pour les résultats du traitement distribué

In [6]:
!mkdir -p ~/myinput
!mkdir -p ~/myoutput

Télachargement du jeu de données dans le fichier <u>purchases.txt</u>

In [29]:
!curl -L -o 'purchases.txt' 'https://drive.google.com/u/0/uc?id=1NS-PSXW8bSNpzFH4XRbtmMnMGhXBdYy6&export=download&confirm=t'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  201M  100  201M    0     0   141M      0  0:00:01  0:00:01 --:--:--  186M


Déplacement du fichier <u>purchases.txt</u> dans le répertoire <myinput>

In [28]:
!mv purchases.txt ~/myinput/purchases.txt

mv: cannot stat 'purchases.txt': No such file or directory


Vérification que les fichiers ont été bien copiés

In [27]:
!ls /usr/myinput

ls: cannot access '/usr/myinput': No such file or directory


Affichage des premiers lignes du fichier. Le format des enregistrement est le suivant:
<table border='1'><tr>
<td>Date</td><td>Heure</td><td>Magasin</td><td>Produit</td><td>Montant</td><td>Moyen_de_paiement</td>
</tr></table>
La tabulation <b>\t</b> est utilisée comme séparateur de colonne ✅

In [ ]:
!head -15  ~/myinput/purchases.txt

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Activité 1

Le travail consiste à utiliser <i>MapReduce</i> avec le langage Python et effectuer un traitement distribué. Notre but est de déterminer le total des achats par magasin en exploitant les données du fichier <purchases.txt>.

Vous devrez implémenter les fonctions <i>Map<i> et <i>Reduce</> du traitement.

Contenu de traitement dans la phase \<map>. <h2>C'est à vous de le faire &#8987;</h2>

In [14]:
#@title
#!/usr/bin/python
import sys
# Mettez vos instructions ...
def mapper(file_path):
  fichier = open(file_path, 'r')
  file_line = fichier.readlines()
  fichier.close()
  result = []

  for line in file_line:
    data = line.strip().split("\t")
    if len(data) == 6:
      date, time, store, item, montant, payment = data
      result.append((store, montant))
      #print(store, "\t", str(montant))
  
  return result


"""result = mapper('/usr/lib/jvm/java-8-openjdk-amd64/fichier/purchases.txt')
for i in range(10):
  print(result[i][0], "\t", result[i][1])"""

'result = mapper(\'/usr/lib/jvm/java-8-openjdk-amd64/fichier/purchases.txt\')\nfor i in range(10):\n  print(result[i][0], "\t", result[i][1])'

Sauvegarde du code de traitement de la phase <map> dans le fichier "/content/map.py"

In [59]:
# All cell codes are stored in a List variable "In"
with open('/content/mapper.py', 'w') as f:
  f.write(In[12]) 
f.close()

Attribution de permission d'accès et d'exécution sur le fichier <mapper.py>

In [55]:
!chmod u+rwx /content/mapper.py

Test du traitement de la phase <map> sur quelques enregistrements

In [ ]:
!head -10 ~/myinput/purchases.txt | python3 /content/mapper.py

Contenu de traitement de la phase \<reduce>. <h2>C'est à vous de le faire aussi &#128521;</h2>

In [15]:
#!/usr/bin/python
from operator import itemgetter
import sys
# Mettez vos instructions ...

def reducer(file_path):
  map = mapper(file_path)
  map_new = {}

  for elm in map:
    if elm[0] not in map_new:
      map_new.update({elm[0] : 0})

  for key in map:
    map_new.update({key[0] : float(map_new[key[0]]) + float(key[1])})

  print("Magazin\t\tMontant Total")
  print("##################################")
  for elm in map_new:
    print(elm, "\t", map_new[elm])


file_path = '/usr/lib/jvm/java-8-openjdk-amd64/fichier/purchases.txt'
reducer(file_path)


"""
salesTotal = 0
oldKey = None
for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) != 2:
    continue
  
  thisKey, thisSale = data
  if oldKey and oldKey != thisKey:
    print(oldKey, "\t", str(salesTotal))
    salesTotal = 0
    
  oldKey = thisKey
  salesTotal += float (thisSale)

if oldKey != None:
  print(oldKey, "\t", str(salesTotal))"""

Magazin		Montant Total
##################################
San Jose 	 9936721.410000037
Fort Worth 	 10120830.650000045
San Diego 	 9966038.390000092
Pittsburgh 	 10090124.820000142
Omaha 	 10026642.340000028
Stockton 	 10006412.640000084
Austin 	 10057158.89999997
New York 	 10085293.549999924
Corpus Christi 	 9976522.769999985
Las Vegas 	 10054257.979999974
Newark 	 10144052.80000001
Greensboro 	 10033781.390000034
San Francisco 	 9995570.540000018
Lincoln 	 10069485.400000012
Buffalo 	 10001941.19000001
Boston 	 10039473.279999953
Houston 	 10042106.270000026
Virginia Beach 	 10086553.499999968
Riverside 	 10006695.420000002
Tulsa 	 10064955.900000058
Reno 	 10079955.159999985
Chicago 	 10062522.06999999
Fort Wayne 	 10132594.019999968
San Bernardino 	 9965152.039999986
Madison 	 10032035.540000046
Portland 	 10007635.769999959
Anchorage 	 9933500.400000054
Spokane 	 10083362.979999933
Fresno 	 9976260.260000074
Aurora 	 9992970.92000003
Philadelphia 	 10190080.25999996
Fremont 	 100

'\nsalesTotal = 0\noldKey = None\nfor line in sys.stdin:\n  data = line.strip().split("\t")\n  if len(data) != 2:\n    continue\n  \n  thisKey, thisSale = data\n  if oldKey and oldKey != thisKey:\n    print(oldKey, "\t", str(salesTotal))\n    salesTotal = 0\n    \n  oldKey = thisKey\n  salesTotal += float (thisSale)\n\nif oldKey != None:\n  print(oldKey, "\t", str(salesTotal))'

Sauvegarde du code de traitement de la phase <reduce> dans le fichier "/content/reduce.py"

In [ ]:
# All cell codes are stored in a List variable "In"
with open('/content/reducer.py', 'w') as f:
  f.write(In[37]) 
f.close()

Attribution de permission d'accès et d'exécution sur le fichier <reducer.py>

In [ ]:
!chmod u+rwx /content/reducer.py

Test du traitement distribué sur quelques enregistrements

In [ ]:
!head -50 ~/myinput/purchases.txt | python3 /content/mapper.py | sort | python3 /content/reducer.py

Lancement d'un job entier. Le résultat est dans le dossier "~/tryout".

In [ ]:
!rm -r ~/myoutput
!/usr/local/hadoop-3.3.0/bin/hadoop jar /usr/local/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -input ~/myinput -output ~/myoutput -file /content/mapper.py  -file /content/reducer.py  -mapper 'python mapper.py'  -reducer 'python reducer.py'

Affichage du contenu du dossier "~/myoutput".

In [ ]:
!ls ~/myoutput

Affichage d'un part du résultat contenu dans le fichier de sortie. On y trouve la fréquence de chaque mot contenu dans le corpus de documents.

In [ ]:
!tail -n 30 ~/myoutput/part-00000

# Activité 2
Nous continuons à travailler avec le même fichier en entrées (purchases.txt), mais pour obtenir des résultats différents. <u>Le but est donc d’écrire vos propres Mappers et Reducers</u>.
<ol>
<li>Donner le nombre de paiement par mode de paiement.</li>
<li>Quel est le chiffre d'affaire réalisé selon les jours de la semaine ?</li>
<li>Quelle est la liste des magasins ?</li>
<li>Quel est le nombre total des ventes et la valeur totale des ventes de tous magasins confondus ?</i>
</ol>

# Références
**Réalisez des calculs distribués sur des données massives** : 
https://openclassrooms.com/fr/courses/4297166-realisez-des-calculs-distribues-sur-des-donnees-massives/4308656-familiarisez-vous-avec-hadoop

**Hadoop : la nouvelle infrastructure de gestion de données** : https://juvenal-chokogoue.developpez.com/tutoriels/hadoop-fonctionnement/

**MapReduce : comment l’utiliser pour le Big Data ?** : https://datascientest.com/mapreduce

**Calcul distribué: Hadoop et MapReduce** : http://b3d.bdpedia.fr/calculdistr.html

**Language Processing and Python** : https://www.nltk.org/book/ch01.html